In [1]:
import numpy as np 
import pandas as pd
import itertools
from scipy.optimize import linprog

In [2]:
#Read the data with the load profile and proces for each hour
data = pd.read_csv('load_price_business_working_hours_winter_summer.csv')
data.drop(columns=['Unnamed: 0'])

num_work_hours = 10

##Variables for sensitivity analysis

# Size of the storage
num_vehicles = [100]

# Battery specifications
charger_power = [0.0072] #MW
energy_vehicle = [0.064] #MWh
eff = [0.97] #eficiency 
avg_initial_energy = [0.7]


In [3]:
len(data)/10

181.0

In [4]:
len(data)/8

226.25

In [5]:
# https://www.geeksforgeeks.org/python-creating-3d-list/
 
def ThreeD(a, b, c): 
    lst = [[ ['0' for col in range(a)] for col in range(b)] for row in range(c)] 

    return lst 

In [6]:
def daily_analysis(num_vehicles, charger_power, energy_vehicle, eff, avg_initial_energy,data):
    
    
    ##Storage specifications

    #Power of storage charge/discharge
    Psc_max = charger_power #MW
    Pdc_max = charger_power #MW

    #Maximum/minimum avaliable energy in the storage system
    Es_max = energy_vehicle #MW
    Es_min = 0.25*energy_vehicle #MW

    #Initial storage
    Initial_battery_vec = np.random.normal(avg_initial_energy,0.1,num_vehicles) 
    
    Initial_E_vec = Initial_battery_vec*energy_vehicle #MWh
       
    ##Optimization

    max_demand = 100/1000 #Max demand is assumed to be 0.1 MW (medium business)
    
    #Arrays for the results
    
    Px = ThreeD(len(data), 1, num_vehicles)
    Pchar = ThreeD(len(data), 1, num_vehicles)
    Pdis = ThreeD(len(data), 1, num_vehicles)
    E = ThreeD(len(data), 1, num_vehicles)
    original_cost = [ ];
    savings = ThreeD(int(len(data)/num_work_hours), 1, num_vehicles) 
    
    
    for b in range(len(Initial_E_vec)): 
        
#         for d in range(int(len(data)/10)):  # For each day (which has 10 labor hours)
            
#             Pd = data['Hourly Load'][d*10:(d+1)*10]/1000; #Demand vector in MWh

        for d in range(int(len(data)/num_work_hours)):  # For each day (which has 10 labor hours)
            
            Pd = data['Hourly Load'][d*num_work_hours:(d+1)*num_work_hours]/1000; #Demand vector in MWh
            
            n = len(Pd)  #Number of hours
            
            # vector x: Px(1 - n) | Pchar(n+1 - 2n) | Pdis(2n+1 - 3n) | Energy(3n+1 - 4n) | E0(4n+1)|
            # Where:
            #   - Px is the exchange with the grid 
            #   - Pchar is the Energy Storage charge
            #   - Pdis is the Energy Storage discharge

            f1 = data['Hourly Price'][d*num_work_hours:(d+1)*num_work_hours]*1000 #prices in $/MWh (original in $/kWh)

            f23 = list(np.zeros(2*n))
            f4 = list(np.zeros(n))

            f = []  #cost vector

            f.append(f1)
            f.append(f23)
            f.append(f4)

            f = list(itertools.chain(*f)) #flatten the vector 
            f.append(0.0)
            
            # Building Aeq Matrix
            # n rows for power balances equations 
            # n rows for energy balances equations
            # 2 rows for setting the initial energy in the storage and final energy
            # equal to initial 
            
            Aeq = np.zeros((2*n+2,4*n+1))
            #indices for submatrices
            sP = 0
            sC = n
            sD = 2*n
            sE = 3*n 
            
            for i in range(n): 
                Aeq[i,i]=1
                Aeq[i,i+sC]=-1
                Aeq[i,i+sD]=1
                
                r = n+i
                Aeq[r,i+sC]=-eff
                Aeq[r,i+sD]=1/eff
                Aeq[r,i+sE]=1
                Aeq[r,i+sE-1]=-1
            
            
            Aeq[n,sE-1]=0
            Aeq[n,4*n]=-1
            Aeq[2*n,4*n]=1
            Aeq[2*n+1,4*n-1]=1
            Aeq[2*n+1,4*n]=-1

            
            #Building Beq Matrix

            beq = []

            beq.append(list(Pd))
            beq.append(list(np.zeros(n+2)))

            beq = list(itertools.chain(*beq)) #flatten the vector 
            
            beq[2*n] = Initial_E_vec[b-1]
            
                        
            # Lower bounds 
            # The minimum exchange with the grid is when there is no demand and the
            # storage system is discharging at maximum rate
            # The minimum charge and discharge power is zero 
            
            lb_vec = []
            lb_vec.append(list(-1*Pdc_max*np.ones(n)))
            lb_vec.append(np.zeros(n))
            lb_vec.append(np.zeros(n))
            lb_vec.append(Es_min*np.ones(n))
            lb_vec = list(itertools.chain(*lb_vec))
            lb_vec.append(0.0)                     
            
            
            # Upper bounds 
            # The maximum exchange with the grid is when there is the maximum demand and the
            # storage system is charging at maximum rate
            # The maximum charge and discharge power are specified in the previous section 

            ub_vec = []
            ub_vec.append(list(Psc_max*np.ones(n)+max_demand))
            ub_vec.append(Psc_max*np.ones(n))
            ub_vec.append(Pdc_max*np.ones(n))
            ub_vec.append(Es_max*np.ones(n))
            ub_vec = list(itertools.chain(*ub_vec))
            ub_vec.append(999)  
            
            bounds = []

            for i in range(len(lb_vec)): 
                bounds.append((lb_vec[i],ub_vec[i]))           

            res = linprog(f, A_eq=Aeq, b_eq=beq, bounds=bounds, options={"disp": True})
 
            original_cost_aux = original_cost
            
            pos_x = 0
            for c in range(d*num_work_hours,(d+1)*num_work_hours):
                Px[b][0][c] = res.x[pos_x]
                Pchar[b][0][c] = res.x[n+pos_x]
                Pdis[b][0][c] = res.x[2*n+pos_x]
                E[b][0][c]= res.x[3*n+pos_x]
                
                pos_x = pos_x+1

            Pdis_curr_iter = res.x[2*n:3*n]
            Pchar_curr_iter = res.x[n:2*n]

            #The original cost just need to be calculated once, 
            #so it is going to be calculated for the first vehicle
            
            if b == 0: 
                original_cost.append(sum(Pd*f1))

            #The reduction associated to the storage must be calculated for each
            #vehicle, as each one is considered as an independent storage
            savings[b][0][d]= [sum(Pdis_curr_iter*f1) - sum(Pchar_curr_iter*f1)] 

            
    ##Economic analysis

    aggregated_savings= np.zeros(int(len(data)/num_work_hours));

    for d in range(int(len(data)/num_work_hours)):
        agg_svg_day = 0
        for b in range(len(Initial_E_vec)):
            agg_svg_aux =  agg_svg_day
            agg_svg_day = agg_svg_aux + float(savings[b][0][d][0])

        aggregated_savings[d] = agg_svg_day

    total_original_cost = sum(original_cost)
    total_savings = sum(aggregated_savings)

    return aggregated_savings, original_cost
            
        

In [7]:
for n in num_vehicles: 
    for c in charger_power: 
        for e in energy_vehicle: 
            for ef in eff: 
                for i in avg_initial_energy: 
                    
                    [aggregated_savings, daily_original_cost] = daily_analysis(n, c, e, ef, i,data)
                    

results = pd.DataFrame(aggregated_savings, columns =['aggregated_savings']) 

results["daily_original_cost"] = daily_original_cost

                    

Optimization terminated successfully.
         Current function value: 38.535324   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.726564   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.148376   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.583758   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.819850   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.332920   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.458294   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.590055   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.460918   
         Iterations: 89
Optimization terminated succ

Optimization terminated successfully.
         Current function value: 38.880804   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.028180   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.071280   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.027727   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.223699   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.302430   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.102366   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.117412   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.160286   
         Iterations: 89
Optimization terminated succ

Optimization terminated successfully.
         Current function value: 40.989728   
         Iterations: 86
Optimization terminated successfully.
         Current function value: 40.998217   
         Iterations: 86
Optimization terminated successfully.
         Current function value: 41.064002   
         Iterations: 86
Optimization terminated successfully.
         Current function value: 41.129802   
         Iterations: 86
Optimization terminated successfully.
         Current function value: 190.960808  
         Iterations: 85
Optimization terminated successfully.
         Current function value: 190.522558  
         Iterations: 85
Optimization terminated successfully.
         Current function value: 191.009942  
         Iterations: 85
Optimization terminated successfully.
         Current function value: 51.188144   
         Iterations: 86
Optimization terminated successfully.
         Current function value: 50.709971   
         Iterations: 86
Optimization terminated succ

Optimization terminated successfully.
         Current function value: 38.954436   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.726364   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.853262   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.512657   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.987020   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.836828   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.591494   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.765693   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.666068   
         Iterations: 89
Optimization terminated succ

Optimization terminated successfully.
         Current function value: 39.764012   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 154.195295  
         Iterations: 84
Optimization terminated successfully.
         Current function value: 39.588595   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.545448   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.690798   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.404896   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.762646   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.328939   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.236376   
         Iterations: 89
Optimization terminated succ

...

Optimization terminated successfully.
         Current function value: 38.954436   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.726364   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.853262   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.512657   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.987020   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.836828   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.591494   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.765693   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.666068   
         Iterations: 89
Optimization terminated succ

Optimization terminated successfully.
         Current function value: 39.514819   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.764012   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 154.649560  
         Iterations: 85
Optimization terminated successfully.
         Current function value: 39.588595   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.545448   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.690798   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.404896   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.762646   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.328939   
         Iterations: 89
Optimization terminated succ

Optimization terminated successfully.
         Current function value: 38.518454   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.491562   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.893174   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.538286   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.198371   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.722771   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.313884   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.847866   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.800618   
         Iterations: 89
Optimization terminated succ

Optimization terminated successfully.
         Current function value: 39.397377   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.435906   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.407497   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.421893   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 38.869999   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.218500   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.227714   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.228034   
         Iterations: 89
Optimization terminated successfully.
         Current function value: 39.044755   
         Iterations: 89
Optimization terminated succ

Optimization terminated successfully.
         Current function value: 495.286752  
         Iterations: 86
Optimization terminated successfully.
         Current function value: 494.113600  
         Iterations: 86
Optimization terminated successfully.
         Current function value: 193.230541  
         Iterations: 86
Optimization terminated successfully.
         Current function value: 192.552193  
         Iterations: 86
Optimization terminated successfully.
         Current function value: 51.061143   
         Iterations: 87
Optimization terminated successfully.
         Current function value: 51.241207   
         Iterations: 87
Optimization terminated successfully.
         Current function value: 51.488470   
         Iterations: 87


In [8]:
results[140:180]


,aggregated_savings,daily_original_cost
140,36.679099,41.503955
141,36.679099,41.467624
142,36.679099,41.346673
143,36.679099,40.850454
144,36.679099,41.423294
145,36.679099,41.493036
146,36.679099,41.466537
147,93.961338,51.860687
148,855.589931,198.134218
149,855.589931,198.782879


In [9]:
results.to_pickle("./results_winter_summer_97.pkl")

In [10]:
def hourly_analysis(num_vehicles, charger_power, energy_vehicle, eff, avg_initial_energy,data):
    
    
    ##Storage specifications

    #Power of storage charge/discharge
    Psc_max = charger_power #MW
    Pdc_max = charger_power #MW

    #Maximum/minimum avaliable energy in the storage system
    Es_max = energy_vehicle #MW
    Es_min = 0.25*energy_vehicle #MW

    #Initial storage
    Initial_battery_vec = np.random.normal(avg_initial_energy,0.1,num_vehicles) 
    
    Initial_E_vec = Initial_battery_vec*energy_vehicle #MWh
       
    ##Optimization

    max_demand = 100/1000 #Max demand is assumed to be 0.1 MW (medium business)
    
    #Arrays for the results
    
    Px = ThreeD(len(data), 1, num_vehicles)
    Pchar = ThreeD(len(data), 1, num_vehicles)
    Pdis = ThreeD(len(data), 1, num_vehicles)
    E= ThreeD(len(data), 1, num_vehicles)
    original_cost = [ ];
    savings = ThreeD(int(len(data)), 1, num_vehicles) 
   

    # For this day (which has 10 labor hours)
    # For each vehicle
    
    for b in range(len(Initial_E_vec)): 
            
        Pd = data['Hourly Load'] #Demand vector in MWh
        
        n = len(Pd)  #Number of hours

        # vector x: Px(1 - n) | Pchar(n+1 - 2n) | Pdis(2n+1 - 3n) | Energy(3n+1 - 4n) | E0(4n+1)|
        # Where:
        #   - Px is the exchange with the grid 
        #   - Pchar is the Energy Storage charge
        #   - Pdis is the Energy Storage discharge

        f1 = data['Hourly Price'] #prices in $/MWh (original in $/kWh)

        f23 = list(np.zeros(2*n))
        f4 = list(np.zeros(n))

        f = []  #cost vector

        f.append(f1)
        f.append(f23)
        f.append(f4)

        f = list(itertools.chain(*f)) #flatten the vector 
        f.append(0.0)

        # Building Aeq Matrix
        # n rows for power balances equations 
        # n rows for energy balances equations
        # 2 rows for setting the initial energy in the storage and final energy
        # equal to initial 

        Aeq = np.zeros((2*n+2,4*n+1))
        #indices for submatrices
        sP = 0
        sC = n
        sD = 2*n
        sE = 3*n 

        for i in range(n): 
            Aeq[i,i]=1
            Aeq[i,i+sC]=-1
            Aeq[i,i+sD]=1

            r = n+i
            Aeq[r,i+sC]=-eff
            Aeq[r,i+sD]=1/eff
            Aeq[r,i+sE]=1
            Aeq[r,i+sE-1]=-1


        Aeq[n,sE-1]=0
        Aeq[n,4*n]=-1
        Aeq[2*n,4*n]=1
        Aeq[2*n+1,4*n-1]=1
        Aeq[2*n+1,4*n]=-1


        #Building Beq Matrix

        beq = []

        beq.append(list(Pd))
        beq.append(list(np.zeros(n+2)))

        beq = list(itertools.chain(*beq)) #flatten the vector 

        beq[2*n] = Initial_E_vec[b-1]


        # Lower bounds 
        # The minimum exchange with the grid is when there is no demand and the
        # storage system is discharging at maximum rate
        # The minimum charge and discharge power is zero 

        lb_vec = []
        lb_vec.append(list(-1*Pdc_max*np.ones(n)))
        lb_vec.append(np.zeros(n))
        lb_vec.append(np.zeros(n))
        lb_vec.append(Es_min*np.ones(n))
        lb_vec = list(itertools.chain(*lb_vec))
        lb_vec.append(0.0)                     


        # Upper bounds 
        # The maximum exchange with the grid is when there is the maximum demand and the
        # storage system is charging at maximum rate
        # The maximum charge and discharge power are specified in the previous section 

        ub_vec = []
        ub_vec.append(list(Psc_max*np.ones(n)+max_demand))
        ub_vec.append(Psc_max*np.ones(n))
        ub_vec.append(Pdc_max*np.ones(n))
        ub_vec.append(Es_max*np.ones(n))
        ub_vec = list(itertools.chain(*ub_vec))
        ub_vec.append(999)  

        bounds = []

        for i in range(len(lb_vec)): 
            bounds.append((lb_vec[i],ub_vec[i]))           

        res = linprog(f, A_eq=Aeq, b_eq=beq, bounds=bounds, options={"disp": True})
        
        print(res)

        original_cost_aux = original_cost

        pos_x = 0
        for c in range(0,num_work_hours):
            Px[b][0][c] = res.x[pos_x]
            Pchar[b][0][c] = res.x[n+pos_x]
            Pdis[b][0][c] = res.x[2*n+pos_x]
            E[b][0][c]= res.x[3*n+pos_x]

            pos_x = pos_x+1

        Pdis_curr_iter = res.x[2*n:3*n]
        Pchar_curr_iter = res.x[n:2*n]

        #The original cost just need to be calculated once, 
        #so it is going to be calculated for the first vehicle

        if b == 0: 
            original_cost.append(Pd*f1)

        #The reduction associated to the storage must be calculated for each
        #vehicle, as each one is considered as an independent storage
        savings[b][0][0] = [Pdis_curr_iter*f1 - Pchar_curr_iter*f1] 
        
            
    ##Economic analysis

    aggregated_savings_list = []
    aggregated_savings = np.zeros(n)

    for b in range(len(Initial_E_vec)):
        agg_svg_aux =  aggregated_savings
        aggregated_savings  = np.add(agg_svg_aux,savings[b][0][0])

    aggregated_savings_list.append(aggregated_savings)
    total_original_cost = sum(original_cost)
    total_savings = sum(aggregated_savings)

    return aggregated_savings_list, original_cost

In [11]:
data_day_high_price = data[0:num_work_hours]
data_day_high_price.reset_index(drop=True)

,Unnamed: 0,Date,Hourly Price,Hourly Load
0,0,2019-02-02 08:00:00,0.05805,30.857190
1,1,2019-02-02 09:00:00,0.05959,31.696820
2,2,2019-02-02 10:00:00,0.06158,39.347903
3,3,2019-02-02 11:00:00,0.06158,65.419697
4,4,2019-02-02 12:00:00,0.06061,76.912258
5,5,2019-02-02 13:00:00,0.06065,80.863899
6,6,2019-02-02 14:00:00,0.05997,82.595389
7,7,2019-02-02 15:00:00,0.05986,76.888351
8,8,2019-02-02 16:00:00,0.06097,74.383391
9,9,2019-02-02 17:00:00,0.06515,73.228656


In [12]:
### Hourly analysis for the day , where prices are extremely high (and the savings as well)

for n in num_vehicles: 
    for c in charger_power: 
        for e in energy_vehicle: 
            for ef in eff: 
                for i in avg_initial_energy: 
                    
                    [aggregated_savings, daily_original_cost] = hourly_analysis(n, c, e, ef, i,data_day_high_price)


Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the required tolerance of 1e-12 for a solution to be considered 'close enough' to zero to be a basic solution. Consider increasing the tolerance to be greater than 6.3e+02. If this tolerance is unacceptably  large the problem may be infeasible.
         Iterations: 69
     con: array([ 3.07499898e+01,  3.15896197e+01,  3.92407029e+01,  6.53124971e+01,
        7.68050584e+01,  8.07566989e+01,  8.24881889e+01,  7.67811513e+01,
        7.42761914e+01,  7.31214558e+01,  0.00000000e+00,  0.00000000e+00,
       -1.38777878e-17,  6.93889390e-18, -2.08166817e-17,  6.93889390e-18,
        0.00000000e+00, -6.93889390e-18, -6.93889390e-18,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00])
     fun: 0.065178672
 message: "Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the requi

Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the required tolerance of 1e-12 for a solution to be considered 'close enough' to zero to be a basic solution. Consider increasing the tolerance to be greater than 6.3e+02. If this tolerance is unacceptably  large the problem may be infeasible.
         Iterations: 71
     con: array([ 3.07499898e+01,  3.15896197e+01,  3.92407029e+01,  6.53124971e+01,
        7.68050584e+01,  8.07566989e+01,  8.24881889e+01,  7.67811513e+01,
        7.42761914e+01,  7.31214558e+01, -6.93889390e-18,  0.00000000e+00,
        6.93889390e-18, -1.38777878e-17,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -6.93889390e-18,  6.93889390e-18, -6.93889390e-18,
        0.00000000e+00,  0.00000000e+00])
     fun: 0.065178672
 message: "Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the requi

Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the required tolerance of 1e-12 for a solution to be considered 'close enough' to zero to be a basic solution. Consider increasing the tolerance to be greater than 6.3e+02. If this tolerance is unacceptably  large the problem may be infeasible.
         Iterations: 69
     con: array([ 3.07499898e+01,  3.15896197e+01,  3.92407029e+01,  6.53124971e+01,
        7.68050584e+01,  8.07566989e+01,  8.24881889e+01,  7.67811513e+01,
        7.42761914e+01,  7.31214558e+01, -6.93889390e-18,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -1.38777878e-17,  6.93889390e-18,
       -6.93889390e-18,  6.93889390e-18,  6.93889390e-18, -6.93889390e-18,
        0.00000000e+00,  6.93889390e-18])
     fun: 0.065178672
 message: "Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the requi

Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the required tolerance of 1e-12 for a solution to be considered 'close enough' to zero to be a basic solution. Consider increasing the tolerance to be greater than 6.3e+02. If this tolerance is unacceptably  large the problem may be infeasible.
         Iterations: 68
     con: array([ 3.07499898e+01,  3.15896197e+01,  3.92407029e+01,  6.53124971e+01,
        7.68050584e+01,  8.07566989e+01,  8.24881889e+01,  7.67811513e+01,
        7.42761914e+01,  7.31214558e+01,  0.00000000e+00,  0.00000000e+00,
       -6.93889390e-18, -6.93889390e-18, -6.93889390e-18,  0.00000000e+00,
       -6.93889390e-18,  8.97764108e-18,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00])
     fun: 0.065178672
 message: "Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the requi

Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the required tolerance of 1e-12 for a solution to be considered 'close enough' to zero to be a basic solution. Consider increasing the tolerance to be greater than 6.3e+02. If this tolerance is unacceptably  large the problem may be infeasible.
         Iterations: 71
     con: array([ 3.07499898e+01,  3.15896197e+01,  3.92407029e+01,  6.53124971e+01,
        7.68050584e+01,  8.07566989e+01,  8.24881889e+01,  7.67811513e+01,
        7.42761914e+01,  7.31214558e+01, -6.93889390e-18,  0.00000000e+00,
        6.93889390e-18, -1.38777878e-17,  0.00000000e+00, -6.93889390e-18,
       -6.93889390e-18, -6.93889390e-18,  6.93889390e-18,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00])
     fun: 0.065178672
 message: "Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the requi

...

Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the required tolerance of 1e-12 for a solution to be considered 'close enough' to zero to be a basic solution. Consider increasing the tolerance to be greater than 6.3e+02. If this tolerance is unacceptably  large the problem may be infeasible.
         Iterations: 68
     con: array([ 3.07499898e+01,  3.15896197e+01,  3.92407029e+01,  6.53124971e+01,
        7.68050584e+01,  8.07566989e+01,  8.24881889e+01,  7.67811513e+01,
        7.42761914e+01,  7.31214558e+01,  0.00000000e+00, -1.38777878e-17,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -1.38777878e-17,  6.93889390e-18,  6.93889390e-18, -6.93889390e-18,
        0.00000000e+00,  0.00000000e+00])
     fun: 0.065178672
 message: "Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the requi

Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the required tolerance of 1e-12 for a solution to be considered 'close enough' to zero to be a basic solution. Consider increasing the tolerance to be greater than 6.3e+02. If this tolerance is unacceptably  large the problem may be infeasible.
         Iterations: 68
     con: array([ 3.07499898e+01,  3.15896197e+01,  3.92407029e+01,  6.53124971e+01,
        7.68050584e+01,  8.07566989e+01,  8.24881889e+01,  7.67811513e+01,
        7.42761914e+01,  7.31214558e+01,  0.00000000e+00,  0.00000000e+00,
       -6.93889390e-18, -6.93889390e-18,  0.00000000e+00, -2.08166817e-17,
        0.00000000e+00,  2.50372460e-19,  1.38777878e-17,  6.93889390e-18,
        0.00000000e+00,  6.93889390e-18])
     fun: 0.065178672
 message: "Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the requi

Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the required tolerance of 1e-12 for a solution to be considered 'close enough' to zero to be a basic solution. Consider increasing the tolerance to be greater than 6.3e+02. If this tolerance is unacceptably  large the problem may be infeasible.
         Iterations: 69
     con: array([ 3.07499898e+01,  3.15896197e+01,  3.92407029e+01,  6.53124971e+01,
        7.68050584e+01,  8.07566989e+01,  8.24881889e+01,  7.67811513e+01,
        7.42761914e+01,  7.31214558e+01, -6.93889390e-18,  0.00000000e+00,
        6.93889390e-18,  6.93889390e-18,  0.00000000e+00, -1.38777878e-17,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  4.64977427e-19,
        0.00000000e+00,  6.93889390e-18])
     fun: 0.065178672
 message: "Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the requi

Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the required tolerance of 1e-12 for a solution to be considered 'close enough' to zero to be a basic solution. Consider increasing the tolerance to be greater than 6.3e+02. If this tolerance is unacceptably  large the problem may be infeasible.
         Iterations: 70
     con: array([ 3.07499898e+01,  3.15896197e+01,  3.92407029e+01,  6.53124971e+01,
        7.68050584e+01,  8.07566989e+01,  8.24881889e+01,  7.67811513e+01,
        7.42761914e+01,  7.31214558e+01,  0.00000000e+00,  0.00000000e+00,
       -6.93889390e-18, -2.08166817e-17,  1.38777878e-17, -5.79433408e-18,
        0.00000000e+00, -6.93889390e-18,  6.93889390e-18,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00])
     fun: 0.065178672
 message: "Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the requi

Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the required tolerance of 1e-12 for a solution to be considered 'close enough' to zero to be a basic solution. Consider increasing the tolerance to be greater than 6.3e+02. If this tolerance is unacceptably  large the problem may be infeasible.
         Iterations: 70
     con: array([ 3.07499898e+01,  3.15896197e+01,  3.92407029e+01,  6.53124971e+01,
        7.68050584e+01,  8.07566989e+01,  8.24881889e+01,  7.67811513e+01,
        7.42761914e+01,  7.31214558e+01, -6.93889390e-18,  6.93889390e-18,
        6.93889390e-18,  0.00000000e+00,  6.93889390e-18, -1.38777878e-17,
       -1.38777878e-17,  6.93889390e-18, -6.93889390e-18,  6.50968397e-18,
        0.00000000e+00,  6.93889390e-18])
     fun: 0.065178672
 message: "Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 6.3e+02 which exceeds the requi

In [13]:
aggregated_savings

[array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 7.20695205e-18, 1.76333167e-17, 3.23982507e-17,
         5.00273607e-17, 1.54833611e-17]])]

In [14]:
daily_original_cost

[0    1.791260
 1    1.888813
 2    2.423044
 3    4.028545
 4    4.661652
 5    4.904395
 6    4.953245
 7    4.602537
 8    4.535155
 9    4.770847
 dtype: float64]

In [15]:
lista = [2,2,2,2]
np.add(lista,np.zeros(4))

array([2., 2., 2., 2.])